# 数字孪生代理 (Digital Twin Agent)

`数字孪生代理 (DigitalTwinAgent)` 是一种感知代理。其核心职责是充当模拟物理对象（如水库或管道）与更广泛的多代理系统之间的桥梁。它在仿真环境中有效地创建了物理资产的“数字孪生”。

## 角色与功能

在现实世界的系统中，您会有物理传感器（例如，水位传感器）来广播它们的读数。在我们的仿真中，`DigitalTwinAgent` 扮演的就是这个传感器的角色。

1.  **封装一个模型 (Wraps a Model):** 它持有一个物理对象模型（例如 `水库 (Reservoir)`、`渠道 (Canal)`）的实例。
2.  **发布状态 (Publishes State):** 在每个仿真时间步，它查询其封装模型的状态。
3.  **广播 (Broadcasts):** 然后，它将此状态在一个指定的主题上发布到 `MessageBus` (消息总线)。

这使得其他代理，例如 `本地控制代理 (LocalControlAgent)`，能够订阅这些状态主题并接收它们做出决策所需的“传感器”数据，而无需直接访问物理模型本身。这促进了一个解耦且可扩展的架构。

## 仿真示例

在这个例子中，我们将创建一个简单的 `水库 (Reservoir)` 模型。然后，我们将用一个 `DigitalTwinAgent` 来封装它。我们还将创建一个“监控站”来监听状态更新。我们将手动推进水库的状态，然后调用代理的 `run()` 方法，这应该会触发一次状态发布，我们的监控站将会收到该发布。

**注意:** 要运行此Notebook，您需要从项目的根目录运行Jupyter服务器。

In [ ]:
from swp.local_agents.perception.digital_twin_agent import DigitalTwinAgent
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.central_coordination.collaboration.message_bus import MessageBus, Message
from swp.core.interfaces import State, Parameters

# 1. 一个模拟的监控站，用于监听状态更新
received_states = []
def monitoring_station_callback(message: Message):
    print(f"监控站收到状态: {message}")
    received_states.append(message)

# 2. 设置消息总线和物理模型
bus = MessageBus()
bus.subscribe('reservoir_A_state_topic', monitoring_station_callback)

reservoir_model = Reservoir(
    name="Reservoir_A",
    initial_state=State(volume=5000, outflow=10),
    parameters=Parameters(surface_area=1000)
)

# 3. 创建数字孪生代理
twin_agent = DigitalTwinAgent(
    agent_id="twin_for_reservoir_A",
    simulated_object=reservoir_model,
    message_bus=bus,
    state_topic='reservoir_A_state_topic'
)

# 4. 模拟一个时间步
print("--- 初始状态发布 ---")
# 在真实的仿真中，一个平台会周期性地调用 run()
twin_agent.run(current_time=0)

print("\n--- 模拟水库的变化 ---")
# 为演示目的，手动更新水库的状态
reservoir_model.set_inflow(20)
reservoir_model.step(action={'outflow': 12}, dt=3600)
print(f"水库的内部状态更新为: {reservoir_model.get_state()}")

print("\n--- 第二次状态发布 ---")
# 代理现在再次运行并发布*新的*状态
twin_agent.run(current_time=3600)

# --- 验证 ---
assert len(received_states) == 2
assert received_states[0]['volume'] == 5000
assert received_states[1]['volume'] > 5000 # 检查更新后的水量是否已接收
print("\n成功验证代理发布了初始和更新后的状态。")